In [127]:
import tensorflow.keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [128]:
train = pd.read_csv("./Train/Train_Data.csv")
test = pd.read_csv("./Test/Test_Data.csv")

In [129]:
test.shape

(360, 80)

In [111]:
test.isnull().sum()

Id                0
MSSubClass        0
MSZoning          0
LotFrontage      67
LotArea           0
                 ..
MiscVal           0
MoSold            0
YrSold            0
SaleType          0
SaleCondition     0
Length: 80, dtype: int64

In [112]:
#Handle the null data

def purify(df):
    df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
    df.drop(['Alley'],axis=1,inplace=True)
    df['BsmtCond'] = df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])
    df['BsmtQual'] = df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
    df['FireplaceQu'] = df['FireplaceQu'].fillna(df['FireplaceQu'].mode()[0])
    df['GarageType'] = df['GarageType'].fillna(df['GarageType'].mode()[0])
    df['GarageFinish'] = df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])
    df['GarageQual'] = df['GarageQual'].fillna(df['GarageQual'].mode()[0])
    df['GarageCond'] = df['GarageCond'].fillna(df['GarageCond'].mode()[0])
    df.drop(['PoolQC','Fence','MiscFeature'],axis=1,inplace=True)
    



In [113]:
purify(train)
purify(test)

In [114]:
train.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
0   1          60       RL         65.0     8450   Pave      Reg         Lvl   
1   2          20       RL         80.0     9600   Pave      Reg         Lvl   
2   3          60       RL         68.0    11250   Pave      IR1         Lvl   
3   4          70       RL         60.0     9550   Pave      IR1         Lvl   
4   5          60       RL         84.0    14260   Pave      IR1         Lvl   

  Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch PoolArea  \
0    AllPub    Inside  ...             0         0           0        0   
1    AllPub       FR2  ...             0         0           0        0   
2    AllPub    Inside  ...             0         0           0        0   
3    AllPub    Corner  ...           272         0           0        0   
4    AllPub       FR2  ...             0         0           0        0   

  MiscVal MoSold  YrSold  SaleType  SaleCondition  SalePrice  
0       0      2    2008        WD         Normal     208500  
1       0      5    2007        WD         Normal     181500  
2       0      9    2008        WD         Normal     223500  
3       0      2    2006        WD        Abnorml     140000  
4       0     12    2008        WD         Normal     250000  

[5 rows x 77 columns]

In [115]:
columns=['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood',
         'Condition2','BldgType','Condition1','HouseStyle','SaleType',
        'SaleCondition','ExterCond',
         'ExterQual','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
        'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Heating','HeatingQC',
         'CentralAir',
         'Electrical','KitchenQual','Functional',
         'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive']

In [116]:
def category_onehot_multcols(multcolumns):
    df_final=final_df
    i=0
    for fields in multcolumns:
        
        print(fields)
        df1=pd.get_dummies(final_df[fields],drop_first=True)
        
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final=df1.copy()
        else:
            
            df_final=pd.concat([df_final,df1],axis=1)
        i=i+1
       
        
    df_final=pd.concat([final_df,df_final],axis=1)
        
    return df_final

In [118]:

final_df=pd.concat([train,test],axis=0)
final_df=category_onehot_multcols(columns)

MSZoning
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition2
BldgType
Condition1
HouseStyle
SaleType
SaleCondition
ExterCond
ExterQual
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive


In [124]:
final_df =final_df.loc[:,~final_df.columns.duplicated()]
final_df.shape

df_Train=final_df.iloc[:1100,:]
df_Test=final_df.iloc[1100:,:]

In [130]:
df_Test.shape
df_Test.drop(['SalePrice'],axis = 1,inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [143]:
X_train = df_Train.drop(['SalePrice'],axis=1).values
Y_train = df_Train['SalePrice'].values

In [157]:
X_train.shape


(1100, 177)

In [152]:
Y_train

array([208500., 181500., 223500., ..., 170000., 128000., 157000.])

In [160]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [164]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout


# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 50, init = 'he_uniform',activation='relu',input_dim = 177))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 25, init = 'he_uniform',activation='relu'))

# Adding the third hidden layer
classifier.add(Dense(output_dim = 50, init = 'he_uniform',activation='relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'he_uniform'))

# Compiling the ANN
classifier.compile(loss=root_mean_squared_error, optimizer='Adamax')

# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, Y_train,validation_split=0.20, batch_size = 10, nb_epoch = 1000)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=177, units=50, kernel_initializer="he_uniform")`
  if sys.path[0] == '':
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=25, kernel_initializer="he_uniform")`
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50, kernel_initializer="he_uniform")`
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, kernel_initializer="he_uniform")`
/Library/Fra

Train on 880 samples, validate on 220 samples
Epoch 1/1000
880/880 [==============================] - 0s 435us/step - loss: nan - val_loss: nan
Epoch 2/1000
880/880 [==============================] - 0s 386us/step - loss: nan - val_loss: nan
Epoch 3/1000
880/880 [==============================] - 0s 281us/step - loss: nan - val_loss: nan
Epoch 4/1000
880/880 [==============================] - 0s 210us/step - loss: nan - val_loss: nan
Epoch 5/1000
880/880 [==============================] - 0s 196us/step - loss: nan - val_loss: nan
Epoch 6/1000
880/880 [==============================] - 0s 270us/step - loss: nan - val_loss: nan
Epoch 7/1000
880/880 [==============================] - 0s 224us/step - loss: nan - val_loss: nan
Epoch 8/1000
880/880 [==============================] - 0s 197us/step - loss: nan - val_loss: nan
Epoch 9/1000
880/880 [==============================] - 0s 404us/step - loss: nan - val_loss: nan
Epoch 10/1000
880/880 [==============================] - 0s 210us/step -

880/880 [==============================] - 0s 185us/step - loss: nan - val_loss: nan
Epoch 84/1000
880/880 [==============================] - 0s 183us/step - loss: nan - val_loss: nan
Epoch 85/1000
880/880 [==============================] - 0s 208us/step - loss: nan - val_loss: nan
Epoch 86/1000
880/880 [==============================] - 0s 190us/step - loss: nan - val_loss: nan
Epoch 87/1000
880/880 [==============================] - 0s 198us/step - loss: nan - val_loss: nan
Epoch 88/1000
880/880 [==============================] - 0s 204us/step - loss: nan - val_loss: nan
Epoch 89/1000
880/880 [==============================] - 0s 183us/step - loss: nan - val_loss: nan
Epoch 90/1000
880/880 [==============================] - 0s 192us/step - loss: nan - val_loss: nan
Epoch 91/1000
880/880 [==============================] - 0s 197us/step - loss: nan - val_loss: nan
Epoch 92/1000
880/880 [==============================] - 0s 188us/step - loss: nan - val_loss: nan
Epoch 93/1000
880/880 [=

KeyboardInterrupt: 

In [170]:
df_Train.shape

(1100, 178)